In [1]:
%%capture
%cd "Compound GRN ENC Analysis/scripts"

In [2]:
from collections import defaultdict
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn.metrics
import sklearn.svm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
import torch_geometric.nn as gnn

# Params
DATA_FOLDER = os.path.join(os.path.abspath(''), '../../data')
RESULTS_FOLDER = os.path.join(os.path.abspath(''), '../results')
PLOTS_FOLDER = os.path.join(os.path.abspath(''), '../plots')

# Style
sns.set_theme(context='talk', style='white', palette='Accent')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [3]:
# TODO
# Consider weight scaling for disease and ctl individually (maybe?  Depends on methodology)

# Conversion

In [4]:
# # Convert from Gerstein to project format
# save_dir = os.path.join(DATA_FOLDER, 'merged_GRNs_v2', 'Subclass', 'ctrl')
# read_dir = os.path.join(save_dir, 'original')
# fnames = os.listdir(read_dir)
# for fname in fnames:
#     data = pd.read_csv(os.path.join(read_dir, fname), delimiter='\t', low_memory=False)
#     data = data.rename(columns={'TG': 'target', 'edgeWeight': 'importance'})[['TF', 'target', 'importance']]
#     data = data.loc[data['importance'] > data['importance'].quantile(.9)]  # Filter to top 10%
#     data.to_csv(os.path.join(save_dir, fname[:-7] + 'merged.csv'), index=False)

# Data

In [5]:
"""
Cohort : Disease : Delimiter
CMC: SCZ : tsv
UCLA_ASD: ASD : csv
Urban_DLPFC: BPD, SCZ : tsv
Subclass: ASD, BPD, SCZ : csv
"""
data_sources = (
    ('CMC', 'SCZ', '\t'),
    ('UCLA_ASD', 'ASD', ','),
    ('Urban_DLPFC', 'BPD', '\t'),
    ('Urban_DLPFC', 'SCZ', '\t'),
    ('Subclass', 'ASD', ','),  # No CTL
    ('Subclass', 'BPD', ','),  # No CTL
    ('Subclass', 'SCZ', ','),  # No CTL
)
group, disease, delimiter = data_sources[6]
use_ctl = False

In [6]:
# Get AD, BPD, and SCZ labels
gene_dir = os.path.join(DATA_FOLDER, 'new_labels')
gene_fnames = [fname for fname in os.listdir(gene_dir) if fname.endswith('.txt')]
gene_lists = {'.'.join(fname.split('.')[:-1]): np.loadtxt(os.path.join(gene_dir, fname), dtype=str) for fname in gene_fnames}
gene_lists['BPD'] = gene_lists.pop('BD')

# Get ASD labels
sfari = pd.read_csv(os.path.join(DATA_FOLDER, 'sfari/SFARI-Gene_genes_01-16-2024release_03-21-2024export.csv'))
gene_score_threshold = -1
sfari = sfari.loc[sfari['gene-score'] > gene_score_threshold]  # Threshold by score
gene_lists['ASD'] = sfari['gene-symbol'].to_numpy()

# Set positive genes
positive_genes = gene_lists[disease]

# Get files for contrast
base_dir = os.path.join(DATA_FOLDER, 'merged_GRNs_v2', group)
disease_folder = os.path.join(base_dir, disease)
control_folder = os.path.join(base_dir, 'ctrl')
grn_fnames = np.sort(list(set(os.listdir(disease_folder)).intersection(set(os.listdir(control_folder)))))

# Get groups and cell-type based on fname
get_cell_type = lambda fname: '_'.join(fname.split('_')[:-1])

# Model

In [7]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim=32):
        super().__init__()

        # Parameters
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim

        # Classification
        self.net = gnn.Sequential('x, edge_index, edge_weight', [
            # Pure weighted GCN
            (gnn.GCNConv(self.input_dim, self.embedding_dim), 'x, edge_index, edge_weight -> x'),
            nn.BatchNorm1d(self.embedding_dim),
            nn.LeakyReLU(),
            # nn.Dropout(.5),

            (gnn.GCNConv(-1, self.embedding_dim // 2), 'x, edge_index, edge_weight -> x'),
            nn.BatchNorm1d(self.embedding_dim // 2),
            nn.LeakyReLU(),
            # nn.Dropout(.5),

            (gnn.GCNConv(-1, 2), 'x, edge_index, edge_weight -> x'),
        ])

        # Embedding weights
        self.embedding_net = nn.Sequential(
            nn.Linear(2 * self.embedding_dim, 1),
            # nn.Linear(self.embedding_dim, 1),
            nn.LeakyReLU(),
        )

    def get_embeddings(self):
        # return torch.matmul(self.net[0].lin.weight.T, self.net[3].lin.weight.T)
        return self.net[0].lin.weight.T

    def forward(self, x, edge_index, edge_weight):
        return self.net(x, edge_index, edge_weight)

    def predict_adjacency(self, idx=None):
        """Embedding loss function"""
        # Get embeddings
        embeddings = self.get_embeddings()

        # Compare embeddings
        predicted_adjacency_matrix = torch.zeros(embeddings.shape[0] if idx is None else len(idx), embeddings.shape[0])
        for iter_num, i in enumerate(range(embeddings.shape[0]) if idx is None else idx):
            x = torch.concat((embeddings[[i]].expand(embeddings.shape[0], -1), embeddings), dim=1)
            predicted_adjacency_matrix[iter_num] = self.embedding_net(x).squeeze(dim=1)
        predicted_adjacency_matrix.fill_diagonal_(0)  # Don't count self-comparisons

        return predicted_adjacency_matrix

# Training

In [8]:
def pd_to_tg(df, positive_genes):
    """Convert pandas dataframe to torch geometric dataset"""
    df = df.rename(columns=dict(target='TG', importance='weight'))
    unique = np.unique(list(df['TF'])+list(df['TG']))
    x = np.eye(unique.shape[0])
    node_dict = {k: v for v, k in enumerate(unique)}
    node_dict_rev = {v: k for k, v in node_dict.items()}
    from_edges = df['TF'].apply(lambda x: node_dict[x]).to_numpy()
    to_edges = df['TG'].apply(lambda x: node_dict[x]).to_numpy()
    edge_index = np.stack((from_edges, to_edges), axis=0)
    edge_weight = df['weight'].to_numpy()

    # Get y data
    # -1 is unknown, 0 is no disease, 1 is disease
    y = np.zeros(x.shape[0])-1  # Assign genes as unknown
    y[[node_dict_rev[i] in positive_genes for i in range(x.shape[0])]] = 1  # Annotate positive genes
    y[np.argwhere(y!=1).flatten()] = 0  # Mark all others as negative

    # Cast to tensors
    x = torch.tensor(x).float()
    edge_index = torch.tensor(edge_index).long()
    edge_weight = torch.tensor(edge_weight).float()
    if 'Cell Type' in df: edge_attr = torch.tensor(edge_attr).float()
    y = torch.tensor(y).long()

    # Torch data
    data = torch_geometric.data.Data(
        x=x,
        edge_index=edge_index,
        edge_weight=edge_weight,
        y=y)

    # Additional attributes
    data.known_mask = y!=-1
    data.adj_mat = torch.zeros(data.x.shape[0], data.x.shape[0])
    for (tf, tg), w in zip(data.edge_index.T, data.edge_weight):
        data.adj_mat[tf, tg] = w
    data.node_dict = node_dict
    data.disease = df['disease'].to_numpy()

    return data

In [9]:
# Seed
np.random.seed(42)
torch.manual_seed(42)

# Parameters
epochs = 101
epoch_cli = 20
folds = 5
embedding_loss_weight = 1e3

# Run files
stats = defaultdict(lambda: [])
for fname in grn_fnames:
    # Load graph
    disease_graph = pd.read_csv(os.path.join(disease_folder, fname), index_col=False, delimiter=delimiter)
    disease_graph['disease'] = True
    if use_ctl:
        control_graph = pd.read_csv(os.path.join(control_folder, fname), index_col=False, delimiter=delimiter)
        control_graph['disease'] = False
        combined_graph = pd.concat((disease_graph, control_graph), axis=0)
    else:
        combined_graph = disease_graph
    graph = pd_to_tg(combined_graph, positive_genes)

    # Run folds
    folds_idx = torch.randperm(graph.x.shape[0]).split(int(np.ceil(graph.x.shape[0]/folds)))
    for fold in range(folds):
        # Get idx
        val_fold = fold
        train_folds = [i for i in range(folds) if i != val_fold]
        graph.train_idx, graph.val_idx = torch.concat([folds_idx[i] for i in train_folds]), folds_idx[val_fold]
        graph.train_mask = torch.zeros(graph.x.shape[0], dtype=torch.bool); graph.train_mask[graph.train_idx] = True
        graph.val_mask = torch.zeros(graph.x.shape[0], dtype=torch.bool); graph.val_mask[graph.val_idx] = True
        graph.train_mask *= graph.known_mask; graph.val_mask *= graph.known_mask

        # Create model
        model = GCN(graph.x.shape[1]).train()
        # Calculate weights
        # weight = torch.unique(graph.y[graph.train_mask], return_counts=True)[1]
        # weight = weight.sum() / weight; weight = weight / weight.sum()
        # Get helpers
        classification_criterion = nn.CrossEntropyLoss()  # weight=weight
        embedding_criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-1)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=.96)

        # CLI
        print(f'{get_cell_type(fname)} - Fold {fold:02d}')
        for epoch in range(epochs):
            # Run for disease and control
            disease_z = model(graph.x, graph.edge_index[:, graph.disease], graph.edge_weight[graph.disease])
            if use_ctl: control_z = model(graph.x, graph.edge_index[:, ~graph.disease], graph.edge_weight[~graph.disease])

            # Get losses
            idx = graph.train_idx
            sub_idx = np.random.choice(graph.train_idx, len(graph.train_idx) // 10, replace=False)
            disease_classification_loss = classification_criterion(disease_z[idx], graph.y[idx])
            if use_ctl: control_classification_loss = classification_criterion(control_z[idx], graph.y[idx])
            embedding_loss = embedding_loss_weight * embedding_criterion(model.predict_adjacency(sub_idx), graph.adj_mat[sub_idx])  # Maybe replace adj_mat with disease or control?
            loss = disease_classification_loss
            if use_ctl: loss += control_classification_loss
            model.net[0].lin.weight.requires_grad = False
            loss.backward()
            model.net[0].lin.weight.requires_grad = True
            embedding_loss.backward()

            # Step
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

            # CLI
            if epoch % epoch_cli == 0:
                # Training
                print(' - '.join([
                    f'epoch {epoch:04d}',
                    f'dl: {disease_classification_loss.detach().item():.3f}',
                    f'ccl: {(control_classification_loss.detach().item() if use_ctl else 0):.3f}',
                    f'el: {embedding_loss.detach().item():.3f}',
                    f'total: {loss.detach().item():.3f}',
                ]), end=' _ ')

                # Validation
                with torch.no_grad():
                    idx = graph.val_idx
                    disease_classification_loss = classification_criterion(disease_z[idx], graph.y[idx])
                    if use_ctl: control_classification_loss = classification_criterion(control_z[idx], graph.y[idx])
                    embedding_loss = embedding_loss_weight * embedding_criterion(model.predict_adjacency(idx), graph.adj_mat[idx])
                    loss = disease_classification_loss
                    if use_ctl: loss += control_classification_loss
                print(' - '.join([
                    f'val dl: {disease_classification_loss.detach().item():.3f}',
                    f'val ccl: {(control_classification_loss.detach().item() if use_ctl else 0):.3f}',
                    f'val el: {embedding_loss.detach().item():.3f}',
                    f'val total: {loss.detach().item():.3f}',
                ]))

        # Finish training
        model.eval()

        # Record
        stats['Cell Type'].append(get_cell_type(fname))
        stats['Fold'].append(fold)

        # Evaluation
        disease_z = model(graph.x, graph.edge_index[:, graph.disease], graph.edge_weight[graph.disease]).detach()
        if use_ctl: control_z = model(graph.x, graph.edge_index[:, ~graph.disease], graph.edge_weight[~graph.disease]).detach()
        pred_prob = F.softmax(disease_z, dim=1)[:, 1]

        # Training
        auprc = sklearn.metrics.average_precision_score(graph.y[graph.train_mask], pred_prob[graph.train_mask])
        # CLI
        print(' - '.join([
            f'auprc: {auprc:.3f}',
        ]), end=' _ ')
        # Record
        stats['AUPRC'].append(auprc)

        # Validation
        auprc = sklearn.metrics.average_precision_score(graph.y[graph.val_mask], pred_prob[graph.val_mask])
        # CLI
        print(' - '.join([
            f'val auprc: {auprc:.3f}',
        ]), end=' _ ')
        # Record
        stats['Validation AUPRC'].append(auprc)

        # Baselines
        print(' - '.join([
            f'base auprc: {graph.y[graph.known_mask].float().mean():.3f}',
        ]))

        # Save predictions
        pred = F.softmax(disease_z, dim=1)[:, 1]
        node_dict_rev = {v: k for k, v in graph.node_dict.items()}
        node_labels = [node_dict_rev[i] for i in range(len(graph.node_dict))]
        df = pd.DataFrame({'gene': node_labels, 'score': pred.numpy(), 'validation': graph.val_mask.numpy(), 'label': graph.y.numpy()})
        df.to_csv(os.path.join(RESULTS_FOLDER, f'{group}_{disease}_{get_cell_type(fname)}_{fold}_prioritized_genes.csv'), index=False)

        # Formatting
        print()

    # CLI
    print()

Chandelier - Fold 00
epoch 0000 - dl: 0.463 - ccl: 0.000 - el: 1.205 - total: 0.463 _ val dl: 0.445 - val ccl: 0.000 - val el: 2.269 - val total: 0.445
epoch 0020 - dl: 0.119 - ccl: 0.000 - el: 1.334 - total: 0.119 _ val dl: 0.063 - val ccl: 0.000 - val el: 0.656 - val total: 0.063
epoch 0040 - dl: 0.115 - ccl: 0.000 - el: 0.709 - total: 0.115 _ val dl: 0.056 - val ccl: 0.000 - val el: 0.585 - val total: 0.056
epoch 0060 - dl: 0.113 - ccl: 0.000 - el: 0.396 - total: 0.113 _ val dl: 0.056 - val ccl: 0.000 - val el: 0.591 - val total: 0.056
epoch 0080 - dl: 0.112 - ccl: 0.000 - el: 0.611 - total: 0.112 _ val dl: 0.056 - val ccl: 0.000 - val el: 0.585 - val total: 0.056
epoch 0100 - dl: 0.111 - ccl: 0.000 - el: 0.816 - total: 0.111 _ val dl: 0.055 - val ccl: 0.000 - val el: 0.563 - val total: 0.055
auprc: 0.068 _ val auprc: 0.038 _ base auprc: 0.023

Chandelier - Fold 01
epoch 0000 - dl: 0.425 - ccl: 0.000 - el: 4.903 - total: 0.425 _ val dl: 0.427 - val ccl: 0.000 - val el: 0.686 - val t

# Performance

In [10]:
# Aggregate results in cell-type files
for fname in grn_fnames:
    df = None
    for fold in range(folds):
        # Read file
        df_join = pd.read_csv(os.path.join(RESULTS_FOLDER, f'{group}_{disease}_{get_cell_type(fname)}_{fold}_prioritized_genes.csv'))
        df_join = df_join.set_index('gene')
        df_join = df_join.rename(columns=lambda s: (s + f'_{fold}') if s not in ('gene', 'label') else s)

        # Escape if first
        if df is None: df = df_join; continue

        # Concat
        df_join = df_join.drop(columns='label')
        df = df.join(df_join)

    # Add mean and std
    scores = df[[f'score_{fold}' for fold in range(folds)]]
    df['mean'] = scores.mean(axis=1)
    df['std'] = scores.std(axis=1)

    # Order
    df = df[['label', 'mean', 'std'] + [f'score_{fold}' for fold in range(folds)] + [f'validation_{fold}' for fold in range(folds)]]

    # Save to file
    df.to_csv(os.path.join(RESULTS_FOLDER, f'{group}_{disease}_{get_cell_type(fname)}_prioritized_genes.csv'))


In [11]:
# Save stats
df = pd.DataFrame(stats)
fname = os.path.join(RESULTS_FOLDER, f'{group}_{disease}_performance.csv')
df.to_csv(fname)  # Save stats
# df = pd.read_csv(fname)  # Load stats

# # Baseline
# # TODO: Fix baseline to be ct-specific
# base_auprc = graph.y[graph.known_mask].float().mean()

# # Create df
# df['AUPRC'] /= float(base_auprc); df['Validation AUPRC'] /= float(base_auprc)
# df = df[['Cell Type', 'Fold'] + ['AUPRC', 'Validation AUPRC']]
# df = df.melt(id_vars=['Cell Type', 'Fold'], value_name='Fold Change', var_name='Statistic')

# # Plot
# fig, ax = plt.subplots(1, 1, figsize=(10, 6))
# sns.boxplot(data=df, x='Cell Type', y='Fold Change', hue='Statistic', ax=ax)
# sns.despine()
# plt.axhline(y=1, color='black', ls='--')
# # plt.ylim(0, 1)
# plt.xticks(rotation=90)
# fig.savefig(os.path.join(PLOTS_FOLDER, f'GCN_Performance_{group}_{disease}.pdf'))